In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from fibsem import utils, acquire, movement, calibration
from fibsem.structures import ImageSettings, BeamType
from pprint import pprint
import matplotlib.pyplot as plt
from fibsem import milling
import numpy as np


In [ ]:
protocol_path = "protocol/protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path, session_path= r"D:\cyril\tin-balls-sweep")

In [ ]:
settings.image.save_path


In [ ]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.beams.electron_beam.optical_mode.value = "Immersion"
microscope.detector.mode.value = "BackscatterElectrons"
settings.system.stage.tilt_flat_to_electron = 0

import numpy as np
microscope.beams.electron_beam.angular_correction.angle.value = np.deg2rad(-38)

# # LUT: inverse


In [ ]:
from autoscript_sdb_microscope_client.enumerations import DetectorMode, DetectorType
# microscope.detector.set_type_mode(type=DetectorType.ETD, mode = DetectorMode.BACKSCATTER_ELECTRONS)

In [ ]:
from fibsem.structures import MicroscopeSettings
from autoscript_sdb_microscope_client import SdbMicroscopeClient


def run_data_collection(microscope: SdbMicroscopeClient, settings: MicroscopeSettings, dwell_times: list[float], step: int):
    
    settings.image.save = True
    settings.image.autocontrast = False

    for i, dwell_time in enumerate(dwell_times):

        settings.image.dwell_time = dwell_time
        print(f"{i} - dwell_time: {dwell_time}")

        # take multiple short dwell time images
        if dwell_time < 2e-6:
            for j in range(7):
                settings.image.label = f"EB_{step}_{dwell_time}_{j}"
                print(i, j, dwell_time, settings.image.label)
                acquire.new_image(microscope, settings.image)
        else:
            settings.image.label = f"EB_{step}_{dwell_time}_0"
            print(i, 0, dwell_time, settings.image.label)
            # take a single high dwell time
            acquire.new_image(microscope, settings.image)
        
        print("-"*50)

In [ ]:

# start wiith cleaning cross section to clean the area
centre_x = 0
centre_y = 0
width = 40e-6
height = 2.5e-6
depth = 10e-6

microscope.imaging.set_active_view(BeamType.ION.value)
microscope.imaging.set_active_device(BeamType.ION.value)
microscope.patterning.clear_patterns()

pattern = microscope.patterning.create_cleaning_cross_section(center_x=centre_x, center_y=centre_y, width=width, height=height, depth=depth)
pattern.scan_direction = "BottomToTop"

In [ ]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.detector.contrast.limits

In [ ]:
state = calibration.get_current_microscope_state(microscope)
# detector contrast brightness
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
contrast = microscope.detector.contrast.value
brightness = microscope.detector.brightness.value

In [ ]:
start_x, end_x = -8e-6, 33e-6
start_y, end_y = -4e-6, -4e-6
depth = 8e-6
offset = 5e-9


N_SLICES = 500
START_IDX = 22
# for i in range(START_IDX, N_SLICES):

microscope.imaging.set_active_view(BeamType.ION.value)
microscope.imaging.set_active_device(BeamType.ION.value)
microscope.patterning.clear_patterns()
pattern = microscope.patterning.create_line(start_x=start_x, start_y=start_y, end_x=end_x, end_y=end_y, depth=depth)
print(pattern)

In [ ]:

settings.image.hfw = 30e-6
settings.image.resolution = "6144x4096"
dwell_times = [0.5e-6, 1e-6, 3e-6, 5e-6, 8e-6, 20e-6]
settings.image.save = True
settings.image.autocontrast = False
settings.image.gamma.enabled = False

start_x, end_x = -8e-6, 33e-6
start_y, end_y = -4e-6, -4e-6
depth = 8e-6
offset = 5e-9

N_SLICES = 500
START_IDX = 0
for i in range(START_IDX, N_SLICES):

    print(f" ---------------- SLICE {i}/{N_SLICES} ---------------- ")
    
    settings.image.dwell_time = 0.2e-6
    settings.image.save = False
    # slice
    if i > START_IDX:
        microscope.imaging.set_active_view(BeamType.ION.value)
        microscope.imaging.set_active_device(BeamType.ION.value)
        microscope.patterning.clear_patterns()
        pattern = microscope.patterning.create_line(start_x=start_x, start_y=start_y, end_x=end_x, end_y=end_y, depth=depth)
        print(pattern)

        # run
        milling.run_milling(microscope, milling_current=5.6e-9)
    
    # view
    calibration.auto_discharge_beam(microscope, settings.image, n_iterations=5)

    settings.image.autocontrast = False
    run_data_collection(microscope, settings, dwell_times, i)

    start_y += offset
    end_y  += offset
    
    # manually adjust working distance
    wd_diff = offset * np.sin(np.deg2rad(38))
    microscope.beams.electron_beam.working_distance.value -= wd_diff #4e-3# 3.995e-3 

    if i % 50 == 0 and i != START_IDX:
        acquire.autocontrast(microscope, BeamType.ELECTRON)


In [ ]:
# microscope.auto_functions.run_auto_focus()
from autoscript_sdb_microscope_client.structures import RunAutoFocusSettings

focus_settings = RunAutoFocusSettings()

microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.auto_functions.run_auto_focus()
microscope.specimen.stage.link()


# microscope.auto_functions.run_auto_stigmator()

In [ ]:
settings.image.hfw = 30e-6
settings.image.resolution = "6144x4096"
dwell_times = [0.3e-6, 0.5e-6, 1e-6, 3e-6, 5e-6, 8e-6]
settings.image.save = True
settings.image.autocontrast = True
settings.image.gamma.enabled = False
settings.image.dwell_time = 0.5e-6
settings.image.label = "4um"
microscope.beams.electron_beam.working_distance.value = 4e-3# 3.995e-3 
acquire.new_image(microscope, settings.image)

In [ ]:
# # dynamic focus : 38

# 4e-3 - (1000*5e-9)

In [ ]:
settings.image.save_path

In [ ]:
# multichem / GIS query

## Tin Ball Data Collection


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire, movement, calibration
from fibsem.structures import ImageSettings, BeamType
from pprint import pprint
import matplotlib.pyplot as plt
from fibsem import milling
import numpy as np
from copy import deepcopy
import pandas as pd
import os


protocol_path = r"C:\Users\Admin\Github\fibsem\example\protocol\protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path, session_path= r"D:\cyril\tin-balls-sweep")

In [ ]:
# settings
microscope.imaging.set_active_device(BeamType.ELECTRON.value)
microscope.imaging.set_active_view(BeamType.ELECTRON.value)
microscope.beams.electron_beam.optical_mode.value = "Immersion"
microscope.detector.mode.value = "BackscatterElectrons"
settings.system.stage.tilt_flat_to_electron = 0


# microscope.beams.electron_beam.angular_correction.angle.value = np.deg2rad(-38)

# 1kv voltage

# # LUT: inverse


In [ ]:
settings.image.save_path

In [ ]:
# pixelsize (nm), # pixelsize = hfw/n_pixels_x
# hfws
n_pixels_x = 3072 
pixel_size = np.array([1, 2, 3, 4, 5, 6, 8, 10, 12, 14, 16, 18, 20])
hfws = pixel_size * n_pixels_x * 1e-9 # nm

print(hfws)

In [ ]:
# beam current, resolution, dwell time
voltages = [2e3]
# beam_currents = [50e-12, 0.1e-9,  0.2e-9, 0.4e-9, 0.8e-9] 
beam_currents = [3.2e-9]
dwell_times = [2e-6]
resolution = "3072x2048"

settings.image.resolution = resolution
settings.image.autocontrast = False
settings.image.gamma.enabled = False
settings.image.save = True

N_IMAGES = 1

counter = 0
all_params = []
for m, voltage in enumerate(voltages):
    microscope.beams.electron_beam.high_voltage.value = voltage
    for i, beam_current in enumerate(beam_currents):

        # change beam current
        microscope.beams.electron_beam.beam_current.value = beam_current
        # microscope.auto_functions.run_auto_stigmator()
        # microscope.auto_functions.run_auto_focus()
        # calibration.auto_discharge_beam(microscope, settings.image)
        input("confirm beam focus / stigmation")

        for j, hfw in enumerate(hfws):

            acquire.autocontrast(microscope, BeamType.ELECTRON)

            for k, dwell_time in enumerate(dwell_times):
                
                for n in range(N_IMAGES):

                    # set image settings
                    label = f"{counter:02d}_sweep_{m}_{i}_{j}_{k}_{n}_{voltage:.2e}_{beam_current:.2e}_{hfw:.2e}_{dwell_time:.2e}"
                    settings.image.hfw = hfw
                    settings.image.dwell_time = dwell_time
                    settings.image.label = label

                    pprint(settings.image.label)

                    params = {"voltage": voltage, 
                        "beam_current": beam_current,
                        "hfw": hfw,
                        "dwell_time": dwell_time,
                        "n_image": n,
                        "label": label,
                        "count": counter,
                        }

                    all_params.append(deepcopy(params))

                    image = acquire.new_image(microscope, settings.image)
                    counter+=1 
    
# save data
df = pd.DataFrame.from_dict(all_params)
df.to_csv(os.path.join(settings.image.save_path, "data.csv"))
display(df)

# image_count.sweep.m.i.j.k.n.voltage.beam_current.hfw.dwell_time
